In [1]:
import  os, glob
import  random, csv
import tensorflow as tf
from matplotlib import pyplot as plt
import pickle
import cv2
from PIL import Image
import numpy as np
import adapt
from adapt.feature_based import DANN
from tensorflow import keras
from keras import layers
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input, Dense, Reshape,Conv2D,MaxPool2D,Flatten,Activation
from tensorflow.keras.optimizers import Adam
from keras.utils.np_utils import to_categorical

# The path of the datasets, use dict format
dataset_path = {"base": "dataset/ccpd/splitted_plates_base", 
                "challenge":"dataset/ccpd/splitted_plates_challenge",
               "db":"dataset/ccpd/splitted_plates_db",
               "fn":"dataset/ccpd/splitted_plates_fn",
               "weather":"dataset/ccpd/splitted_plates_weather"}
# The path of the saving model check points
save_check_pt = './checkpoints_DANN'

def load_csv(root, filename, name2label):
    # From csv file return images dir,labels list
    if not os.path.exists(os.path.join(root, filename)):
        images = []
        for name in name2label.keys(): 
            images += glob.glob(os.path.join(root, name, '*.png'))
            images += glob.glob(os.path.join(root, name, '*.jpg'))
            images += glob.glob(os.path.join(root, name, '*.jpeg'))
        #print(len(images), images)
        random.shuffle(images) # shuffle images
        with open(os.path.join(root, filename), mode='w', newline='') as f:
            writer = csv.writer(f)
            for img in images:  
                name = img.split(os.sep)[-2]
                label = name2label[name]
                writer.writerow([img, label])
            print('written into csv file:', filename)
    # read existed csv
    images, labels = [], []
    with open(os.path.join(root, filename)) as f:
        reader = csv.reader(f)
        for row in reader:
            img, label = row
            label = int(label)
            images.append(img)
            labels.append(label) 
    # return img dir and label
    return images, labels

def load_ccpd(root, mode='train'):
    name2label = {}  # "sq...":0
    # iterate sub dir, sort, while keep mapping
    for name in sorted(os.listdir(os.path.join(root))):
        # skip non file folder
        if not os.path.isdir(os.path.join(root, name)):
            continue
        # give each category a number
        name2label[name] = len(name2label.keys())
    images, labels = load_csv(root, 'images.csv', name2label)
    if mode == 'train':  # 20%
        images = images[:int(0.2 * len(images))]
        labels = labels[:int(0.2 * len(labels))]
    elif mode == 'val':  # 20% = 60%->80%
        images = images[int(0.6 * len(images)):int(0.8 * len(images))]
        labels = labels[int(0.6 * len(labels)):int(0.8 * len(labels))]
    else:  # all
        images = images
        labels = labels
    return images, labels, name2label

def readCcpdImg(images_dir):
    X = []
    for img_dir in images_dir:
        img = cv2.imread(img_dir)
        img = cv2.resize(img,(16,32))
        X.append(img)
    X = np.array(X)
    return X

# def readCcpdImg(images_dir):
#     X = []
#     for img_dir in images_dir:
#         img = Image.open(img_dir)
#         img = img.convert('L') # conver to grayscale images
#         img = img.resize([16, 32])
#         img_np = np.asarray(img)
#         X.append(img_np.reshape([-1]))
#     X = np.array(X)
#     return X

def get_img_label(path, mode,num_classes=35):
    images_dir, labels, table = load_ccpd(dataset_path[path],mode=mode)
    images = readCcpdImg(images_dir)
    labels = np.array(labels)
    labels = to_categorical(labels, num_classes=num_classes)
    return images,labels

def get_encoder(input_shape=(32,16,3)):
    model = Sequential()
    model.add(Conv2D(32, kernel_size=[3, 3], padding="same", activation='relu',input_shape=input_shape))
    model.add(MaxPool2D(pool_size=[2, 2], strides=2, padding='same'))
    model.add(Conv2D(48, kernel_size=[3, 3], padding="same", activation='relu'))
    model.add(MaxPool2D(pool_size=[2, 2], strides=2, padding='same'))
    model.add(Flatten())
    model.add(Activation('sigmoid'))
    return model

def get_task(input_shape=(1536,)):
    model = Sequential()
    model.add(Dense(1024, activation='relu',input_shape=input_shape))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(34, activation="softmax"))
    return model

def get_discriminator(input_shape=(1536,)):
    model = Sequential()
    model.add(Dense(1024, activation='relu',input_shape=input_shape))
    model.add(Dense(512, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    return model

base_images,base_labels = get_img_label('base', 'all', num_classes=34)
weather_images,weather_labels = get_img_label('weather', 'all', num_classes=34)
weather_images_t,weather_labels_t = get_img_label('weather', 'train', num_classes=34)
challenge_images,challenge_labels = get_img_label('challenge', 'all', num_classes=34)
challenge_images_t,challenge_labels_t = get_img_label('challenge', 'train', num_classes=34)
db_images,db_labels = get_img_label('db', 'all', num_classes=34)
db_images_t,db_labels_t = get_img_label('db', 'train', num_classes=34)
fn_images,fn_labels = get_img_label('fn', 'all', num_classes=34)
fn_images_t,fn_labels_t = get_img_label('fn', 'train', num_classes=34)
print(np.shape(base_images))
#model = get_encoder()
#model.build(input_shape=[None, 32, 16, 3])
#model.summary()
print("load data finished!")

(8446, 32, 16, 3)
load data finished!


In [3]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
print(np.shape(base_images))
print(np.shape(base_labels))
print(np.shape(weather_images_t))
print(np.shape(weather_labels_t))
print(np.shape(weather_images))
print(np.shape(weather_labels))

weather_model = DANN(get_encoder(), get_task(), get_discriminator(),
             lambda_=0.1, optimizer=Adam(0.0001),loss='CategoricalCrossentropy',metrics=["acc"],random_state=0)
weather_model.fit(base_images, base_labels,weather_images_t,weather_labels_t, epochs=86,verbose=1,
                  batch_size = 32,validation_data=(weather_images, weather_labels))
weather_model.score(base_images, base_labels)
weather_model.score(weather_images, weather_labels)
weather_model.save_weights(save_check_pt+ '/weather')

(8446, 32, 16, 3)
(8446, 34)
(762, 32, 16, 3)
(762, 34)
(3812, 32, 16, 3)
(3812, 34)
Epoch 1/86
286/286 [==============================] - 3s 7ms/step - loss: 2.4887 - acc: 0.3724 - disc_loss: 1.1613 - disc_acc: 0.7014 - val_loss: 2.1610 - val_acc: 0.3927
Epoch 2/86
286/286 [==============================] - 2s 6ms/step - loss: 1.5385 - acc: 0.6183 - disc_loss: 1.0674 - disc_acc: 0.7355 - val_loss: 1.3717 - val_acc: 0.6645
Epoch 3/86
286/286 [==============================] - 2s 7ms/step - loss: 1.0035 - acc: 0.7410 - disc_loss: 1.0167 - disc_acc: 0.7487 - val_loss: 0.9908 - val_acc: 0.7411
Epoch 4/86
286/286 [==============================] - 2s 6ms/step - loss: 0.7682 - acc: 0.7957 - disc_loss: 0.9793 - disc_acc: 0.7625 - val_loss: 0.8267 - val_acc: 0.7878
Epoch 5/86
286/286 [==============================] - 2s 6ms/step - loss: 0.6273 - acc: 0.8362 - disc_loss: 0.9544 - disc_acc: 0.7678 - val_loss: 0.7075 - val_acc: 0.8111
Epoch 6/86
286/286 [==============================] - 2s 6ms

In [2]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
print(np.shape(base_images))
print(np.shape(base_labels))
print(np.shape(challenge_images_t))
print(np.shape(challenge_labels_t))
print(np.shape(challenge_images))
print(np.shape(challenge_labels))
challenge_model = DANN(get_encoder(), get_task(), get_discriminator(),
             lambda_=0.1, optimizer=Adam(0.0001),loss='CategoricalCrossentropy',metrics=["acc"],random_state=0)
challenge_model.fit(base_images, base_labels,challenge_images_t,challenge_labels_t,
                    epochs=499,verbose=1,batch_size = 32,validation_data=(challenge_images, challenge_labels))
challenge_model.score(base_images, base_labels)
challenge_model.score(challenge_images, challenge_labels)
challenge_model.save_weights(save_check_pt+ '/challenge')

(8446, 32, 16, 3)
(8446, 34)
(109, 32, 16, 3)
(109, 34)
(548, 32, 16, 3)
(548, 34)
Epoch 1/499
266/266 [==============================] - 3s 6ms/step - loss: 2.4547 - acc: 0.3809 - disc_loss: 0.8779 - disc_acc: 0.7900 - val_loss: 2.4192 - val_acc: 0.3741
Epoch 2/499
266/266 [==============================] - 2s 6ms/step - loss: 1.4336 - acc: 0.6538 - disc_loss: 0.5582 - disc_acc: 0.8855 - val_loss: 1.9135 - val_acc: 0.4872
Epoch 3/499
266/266 [==============================] - 1s 6ms/step - loss: 0.9418 - acc: 0.7538 - disc_loss: 0.4288 - disc_acc: 0.9149 - val_loss: 1.5706 - val_acc: 0.5712
Epoch 4/499
266/266 [==============================] - 1s 6ms/step - loss: 0.7251 - acc: 0.8077 - disc_loss: 0.3093 - disc_acc: 0.9472 - val_loss: 1.2978 - val_acc: 0.6442
Epoch 5/499
266/266 [==============================] - 1s 5ms/step - loss: 0.5752 - acc: 0.8484 - disc_loss: 0.2652 - disc_acc: 0.9565 - val_loss: 1.1665 - val_acc: 0.6971
Epoch 6/499
266/266 [==============================] - 2s

Epoch 48/499
266/266 [==============================] - 2s 6ms/step - loss: 0.0449 - acc: 0.9886 - disc_loss: 0.0815 - disc_acc: 0.9877 - val_loss: 1.3640 - val_acc: 0.7609
Epoch 49/499
266/266 [==============================] - 2s 6ms/step - loss: 0.0220 - acc: 0.9961 - disc_loss: 0.0173 - disc_acc: 0.9975 - val_loss: 0.8801 - val_acc: 0.8504
Epoch 50/499
266/266 [==============================] - 2s 6ms/step - loss: 0.0233 - acc: 0.9955 - disc_loss: 0.0191 - disc_acc: 0.9973 - val_loss: 0.8198 - val_acc: 0.8467
Epoch 51/499
266/266 [==============================] - 2s 6ms/step - loss: 0.0299 - acc: 0.9938 - disc_loss: 0.0579 - disc_acc: 0.9908 - val_loss: 0.8524 - val_acc: 0.8595
Epoch 52/499
266/266 [==============================] - 2s 6ms/step - loss: 0.0346 - acc: 0.9916 - disc_loss: 0.0355 - disc_acc: 0.9946 - val_loss: 0.9629 - val_acc: 0.8212
Epoch 53/499
266/266 [==============================] - 2s 6ms/step - loss: 0.0398 - acc: 0.9902 - disc_loss: 0.0501 - disc_acc: 0.9929

266/266 [==============================] - 2s 6ms/step - loss: 0.0065 - acc: 0.9996 - disc_loss: 0.0350 - disc_acc: 0.9939 - val_loss: 1.3716 - val_acc: 0.8011
Epoch 96/499
266/266 [==============================] - 2s 6ms/step - loss: 0.0606 - acc: 0.9835 - disc_loss: 0.0315 - disc_acc: 0.9949 - val_loss: 0.8964 - val_acc: 0.8285
Epoch 97/499
266/266 [==============================] - 2s 6ms/step - loss: 0.0177 - acc: 0.9971 - disc_loss: 0.0205 - disc_acc: 0.9970 - val_loss: 0.9959 - val_acc: 0.8449
Epoch 98/499
266/266 [==============================] - 2s 6ms/step - loss: 0.0133 - acc: 0.9986 - disc_loss: 0.0094 - disc_acc: 0.9985 - val_loss: 1.0552 - val_acc: 0.8540
Epoch 99/499
266/266 [==============================] - 2s 6ms/step - loss: 0.0178 - acc: 0.9968 - disc_loss: 0.0382 - disc_acc: 0.9943 - val_loss: 1.6279 - val_acc: 0.7901
Epoch 100/499
266/266 [==============================] - 2s 6ms/step - loss: 0.0179 - acc: 0.9959 - disc_loss: 0.0114 - disc_acc: 0.9985 - val_loss:

266/266 [==============================] - 2s 6ms/step - loss: 0.0115 - acc: 0.9986 - disc_loss: 0.0045 - disc_acc: 0.9994 - val_loss: 0.9926 - val_acc: 0.8558
Epoch 143/499
266/266 [==============================] - 2s 6ms/step - loss: 0.0068 - acc: 0.9994 - disc_loss: 0.0042 - disc_acc: 0.9994 - val_loss: 1.0131 - val_acc: 0.8540
Epoch 144/499
266/266 [==============================] - 2s 6ms/step - loss: 0.0079 - acc: 0.9995 - disc_loss: 0.0059 - disc_acc: 0.9989 - val_loss: 1.0196 - val_acc: 0.8522
Epoch 145/499
266/266 [==============================] - 2s 6ms/step - loss: 0.0060 - acc: 0.9996 - disc_loss: 0.0136 - disc_acc: 0.9978 - val_loss: 1.0532 - val_acc: 0.8558
Epoch 146/499
266/266 [==============================] - 2s 6ms/step - loss: 0.0470 - acc: 0.9879 - disc_loss: 0.0521 - disc_acc: 0.9914 - val_loss: 1.0362 - val_acc: 0.8449
Epoch 147/499
266/266 [==============================] - 2s 6ms/step - loss: 0.0173 - acc: 0.9951 - disc_loss: 0.0126 - disc_acc: 0.9981 - val_l

266/266 [==============================] - 2s 6ms/step - loss: 0.0077 - acc: 0.9995 - disc_loss: 0.0043 - disc_acc: 0.9994 - val_loss: 0.8180 - val_acc: 0.8923
Epoch 190/499
266/266 [==============================] - 2s 6ms/step - loss: 0.0182 - acc: 0.9961 - disc_loss: 0.0447 - disc_acc: 0.9922 - val_loss: 1.0781 - val_acc: 0.8266
Epoch 191/499
266/266 [==============================] - 2s 6ms/step - loss: 0.0292 - acc: 0.9918 - disc_loss: 0.0256 - disc_acc: 0.9960 - val_loss: 0.7368 - val_acc: 0.8777
Epoch 192/499
266/266 [==============================] - 2s 6ms/step - loss: 0.0187 - acc: 0.9964 - disc_loss: 0.0013 - disc_acc: 0.9999 - val_loss: 0.7473 - val_acc: 0.8777
Epoch 193/499
266/266 [==============================] - 2s 6ms/step - loss: 0.0095 - acc: 0.9991 - disc_loss: 7.1059e-04 - disc_acc: 0.9999 - val_loss: 1.0613 - val_acc: 0.8668
Epoch 194/499
266/266 [==============================] - 2s 6ms/step - loss: 0.0080 - acc: 0.9995 - disc_loss: 2.8912e-04 - disc_acc: 1.0000

...

Epoch 283/499
266/266 [==============================] - 2s 6ms/step - loss: 0.0284 - acc: 0.9922 - disc_loss: 0.0527 - disc_acc: 0.9911 - val_loss: 0.8515 - val_acc: 0.8704
Epoch 284/499
266/266 [==============================] - 2s 6ms/step - loss: 0.0139 - acc: 0.9978 - disc_loss: 0.0142 - disc_acc: 0.9982 - val_loss: 0.8606 - val_acc: 0.8686
Epoch 285/499
266/266 [==============================] - 2s 6ms/step - loss: 0.0072 - acc: 0.9991 - disc_loss: 0.0096 - disc_acc: 0.9982 - val_loss: 1.0180 - val_acc: 0.8613
Epoch 286/499
266/266 [==============================] - 2s 6ms/step - loss: 0.0089 - acc: 0.9992 - disc_loss: 0.0309 - disc_acc: 0.9955 - val_loss: 0.8355 - val_acc: 0.8777
Epoch 287/499
266/266 [==============================] - 2s 6ms/step - loss: 0.0083 - acc: 0.9992 - disc_loss: 0.0084 - disc_acc: 0.9987 - val_loss: 0.8453 - val_acc: 0.8832
Epoch 288/499
266/266 [==============================] - 2s 6ms/step - loss: 0.0080 - acc: 0.9995 - disc_loss: 0.0153 - disc_acc: 

266/266 [==============================] - 2s 6ms/step - loss: 0.0109 - acc: 0.9973 - disc_loss: 0.0114 - disc_acc: 0.9981 - val_loss: 1.1126 - val_acc: 0.8522
Epoch 331/499
266/266 [==============================] - 2s 6ms/step - loss: 0.0064 - acc: 0.9995 - disc_loss: 0.0037 - disc_acc: 0.9996 - val_loss: 0.9641 - val_acc: 0.8923
Epoch 332/499
266/266 [==============================] - 2s 6ms/step - loss: 0.0039 - acc: 0.9998 - disc_loss: 0.0228 - disc_acc: 0.9963 - val_loss: 0.9307 - val_acc: 0.8887
Epoch 333/499
266/266 [==============================] - 2s 6ms/step - loss: 0.0061 - acc: 0.9996 - disc_loss: 0.0239 - disc_acc: 0.9970 - val_loss: 0.9081 - val_acc: 0.8704
Epoch 334/499
266/266 [==============================] - 2s 6ms/step - loss: 0.0258 - acc: 0.9945 - disc_loss: 0.0395 - disc_acc: 0.9933 - val_loss: 0.8869 - val_acc: 0.8686
Epoch 335/499
266/266 [==============================] - 2s 6ms/step - loss: 0.0085 - acc: 0.9995 - disc_loss: 0.0061 - disc_acc: 0.9992 - val_l

266/266 [==============================] - 2s 6ms/step - loss: 0.0076 - acc: 0.9995 - disc_loss: 8.6248e-06 - disc_acc: 1.0000 - val_loss: 1.1376 - val_acc: 0.8942
Epoch 377/499
266/266 [==============================] - 2s 6ms/step - loss: 0.0076 - acc: 0.9995 - disc_loss: 6.7159e-06 - disc_acc: 1.0000 - val_loss: 1.1347 - val_acc: 0.8905
Epoch 378/499
266/266 [==============================] - 2s 6ms/step - loss: 0.0076 - acc: 0.9995 - disc_loss: 9.8908e-06 - disc_acc: 1.0000 - val_loss: 1.1638 - val_acc: 0.8887
Epoch 379/499
266/266 [==============================] - 2s 6ms/step - loss: 0.0076 - acc: 0.9995 - disc_loss: 1.0366e-05 - disc_acc: 1.0000 - val_loss: 1.1404 - val_acc: 0.8923
Epoch 380/499
266/266 [==============================] - 2s 6ms/step - loss: 0.0655 - acc: 0.9855 - disc_loss: 0.0988 - disc_acc: 0.9864 - val_loss: 0.7820 - val_acc: 0.8978
Epoch 381/499
266/266 [==============================] - 2s 6ms/step - loss: 0.0072 - acc: 0.9994 - disc_loss: 0.0084 - disc_acc

266/266 [==============================] - 2s 6ms/step - loss: 0.0136 - acc: 0.9985 - disc_loss: 0.0073 - disc_acc: 0.9989 - val_loss: 0.7282 - val_acc: 0.8814
Epoch 424/499
266/266 [==============================] - 2s 6ms/step - loss: 0.0059 - acc: 0.9996 - disc_loss: 0.0082 - disc_acc: 0.9990 - val_loss: 0.6997 - val_acc: 0.8850
Epoch 425/499
266/266 [==============================] - 2s 6ms/step - loss: 0.0280 - acc: 0.9948 - disc_loss: 0.0296 - disc_acc: 0.9956 - val_loss: 1.3326 - val_acc: 0.7920
Epoch 426/499
266/266 [==============================] - 2s 6ms/step - loss: 0.0125 - acc: 0.9974 - disc_loss: 0.0097 - disc_acc: 0.9987 - val_loss: 0.7095 - val_acc: 0.8978
Epoch 427/499
266/266 [==============================] - 2s 6ms/step - loss: 0.0127 - acc: 0.9984 - disc_loss: 0.0055 - disc_acc: 0.9992 - val_loss: 0.7377 - val_acc: 0.8741
Epoch 428/499
266/266 [==============================] - 2s 6ms/step - loss: 0.0081 - acc: 0.9994 - disc_loss: 0.0219 - disc_acc: 0.9968 - val_l

266/266 [==============================] - 2s 6ms/step - loss: 0.0121 - acc: 0.9976 - disc_loss: 0.0305 - disc_acc: 0.9956 - val_loss: 1.5232 - val_acc: 0.7427
Epoch 471/499
266/266 [==============================] - 2s 6ms/step - loss: 0.0137 - acc: 0.9974 - disc_loss: 0.0069 - disc_acc: 0.9990 - val_loss: 0.8711 - val_acc: 0.8777
Epoch 472/499
266/266 [==============================] - 2s 6ms/step - loss: 0.0169 - acc: 0.9955 - disc_loss: 7.4526e-04 - disc_acc: 1.0000 - val_loss: 0.7750 - val_acc: 0.8960
Epoch 473/499
266/266 [==============================] - 2s 6ms/step - loss: 0.0069 - acc: 0.9993 - disc_loss: 5.2552e-04 - disc_acc: 1.0000 - val_loss: 0.6729 - val_acc: 0.8905
Epoch 474/499
266/266 [==============================] - 2s 6ms/step - loss: 0.0077 - acc: 0.9995 - disc_loss: 3.4726e-04 - disc_acc: 1.0000 - val_loss: 0.6917 - val_acc: 0.9015
Epoch 475/499
266/266 [==============================] - 2s 6ms/step - loss: 0.0076 - acc: 0.9995 - disc_loss: 2.1409e-04 - disc_acc

In [17]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
print(np.shape(base_images))
print(np.shape(base_labels))
print(np.shape(db_images_t))
print(np.shape(db_labels_t))
print(np.shape(db_images))
print(np.shape(db_labels))
db_model = DANN(get_encoder(), get_task(), get_discriminator(),
             lambda_=0.1, optimizer=Adam(0.0001),loss='CategoricalCrossentropy',metrics=["acc"],random_state=0)
db_model.fit(base_images, base_labels,db_images_t,db_labels_t, epochs=75,
             verbose=1,batch_size = 32,validation_data=(db_images, db_labels))
db_model.score(base_images, base_labels)
db_model.score(db_images, db_labels)
db_model.save_weights(save_check_pt+ '/db')

(8446, 32, 16, 3)
(8446, 34)
(373, 32, 16, 3)
(373, 34)
(1869, 32, 16, 3)
(1869, 34)
Epoch 1/75
269/269 [==============================] - 3s 7ms/step - loss: 2.5443 - acc: 0.3537 - disc_loss: 0.8921 - disc_acc: 0.8052 - val_loss: 2.2829 - val_acc: 0.4452
Epoch 2/75
269/269 [==============================] - 2s 6ms/step - loss: 1.6853 - acc: 0.5856 - disc_loss: 0.9085 - disc_acc: 0.7903 - val_loss: 1.7879 - val_acc: 0.5543
Epoch 3/75
269/269 [==============================] - 2s 7ms/step - loss: 1.1820 - acc: 0.6931 - disc_loss: 0.9568 - disc_acc: 0.7736 - val_loss: 1.5569 - val_acc: 0.5805
Epoch 4/75
269/269 [==============================] - 2s 6ms/step - loss: 0.9379 - acc: 0.7497 - disc_loss: 0.9612 - disc_acc: 0.7730 - val_loss: 1.2984 - val_acc: 0.6790
Epoch 5/75
269/269 [==============================] - 2s 6ms/step - loss: 0.7875 - acc: 0.7876 - disc_loss: 0.9423 - disc_acc: 0.7747 - val_loss: 1.2078 - val_acc: 0.6891
Epoch 6/75
269/269 [==============================] - 2s 6ms

In [5]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
print(np.shape(base_images))
print(np.shape(base_labels))
print(np.shape(fn_images_t))
print(np.shape(fn_labels_t))
print(np.shape(fn_images))
print(np.shape(fn_labels))
fn_model = DANN(get_encoder(), get_task(), get_discriminator(),
             lambda_=0.1, optimizer=Adam(0.0001),loss='CategoricalCrossentropy',metrics=["acc"],random_state=0)
fn_model.fit(base_images, base_labels,fn_images_t,fn_labels_t, epochs=60,verbose=1,
             batch_size = 32,validation_data=(fn_images, fn_labels))
fn_model.score(base_images, base_labels)
fn_model.score(fn_images, fn_labels)
fn_model.save_weights(save_check_pt+ '/fn')

(8446, 32, 16, 3)
(8446, 34)
(48, 32, 16, 3)
(48, 34)
(243, 32, 16, 3)
(243, 34)
Epoch 1/60
264/264 [==============================] - 3s 7ms/step - loss: 2.5634 - acc: 0.3475 - disc_loss: 0.9835 - disc_acc: 0.7653 - val_loss: 2.2690 - val_acc: 0.4527
Epoch 2/60
264/264 [==============================] - 2s 6ms/step - loss: 1.6982 - acc: 0.5868 - disc_loss: 0.7798 - disc_acc: 0.8255 - val_loss: 1.5979 - val_acc: 0.5679
Epoch 3/60
264/264 [==============================] - 2s 6ms/step - loss: 1.1712 - acc: 0.6941 - disc_loss: 0.6733 - disc_acc: 0.8565 - val_loss: 1.2043 - val_acc: 0.6708
Epoch 4/60
264/264 [==============================] - 2s 6ms/step - loss: 0.9244 - acc: 0.7564 - disc_loss: 0.5387 - disc_acc: 0.8946 - val_loss: 1.0022 - val_acc: 0.7037
Epoch 5/60
264/264 [==============================] - 2s 6ms/step - loss: 0.7720 - acc: 0.7920 - disc_loss: 0.4310 - disc_acc: 0.9208 - val_loss: 0.7826 - val_acc: 0.8066
Epoch 6/60
264/264 [==============================] - 2s 6ms/ste